In [2]:
%AddJar http://central.maven.org/maven2/com/google/code/gson/gson/2.2.4/gson-2.2.4.jar -f
%AddJar https://repo.eclipse.org/content/repositories/paho-releases/org/eclipse/paho/org.eclipse.paho.client.mqttv3/1.1.1/org.eclipse.paho.client.mqttv3-1.1.1.jar -f
%AddJar https://github.com/sathipal/spark-streaming-mqtt-with-security_2.10-1.3.0/releases/download/0.0.1/spark-streaming-mqtt-security_2.10-1.3.0-0.0.1.jar -f 
%AddJar http://central.maven.org/maven2/org/scalanlp/breeze_2.10/0.12/breeze_2.10-0.12.jar -f
%AddJar http://central.maven.org/maven2/com/datastax/cassandra/cassandra-driver-core/3.0.0-rc1/cassandra-driver-core-3.0.0-rc1.jar -f
%AddJar http://dl.bintray.com/spark-packages/maven/datastax/spark-cassandra-connector/1.6.0-s_2.10/spark-cassandra-connector-1.6.0-s_2.10.jar -f

Starting download from http://central.maven.org/maven2/com/google/code/gson/gson/2.2.4/gson-2.2.4.jar
Finished download of gson-2.2.4.jar
Starting download from https://repo.eclipse.org/content/repositories/paho-releases/org/eclipse/paho/org.eclipse.paho.client.mqttv3/1.1.1/org.eclipse.paho.client.mqttv3-1.1.1.jar
Finished download of org.eclipse.paho.client.mqttv3-1.1.1.jar
Starting download from https://github.com/sathipal/spark-streaming-mqtt-with-security_2.10-1.3.0/releases/download/0.0.1/spark-streaming-mqtt-security_2.10-1.3.0-0.0.1.jar
Finished download of spark-streaming-mqtt-security_2.10-1.3.0-0.0.1.jar
Starting download from http://central.maven.org/maven2/org/scalanlp/breeze_2.10/0.12/breeze_2.10-0.12.jar
Finished download of breeze_2.10-0.12.jar
Starting download from http://central.maven.org/maven2/com/datastax/cassandra/cassandra-driver-core/3.0.0-rc1/cassandra-driver-core-3.0.0-rc1.jar
Finished download of cassandra-driver-core-3.0.0-rc1.jar
Starting download from http

In [3]:
import scala.math.BigDecimal
import scala.io.Source
import org.eclipse.paho.client.mqttv3._
import org.eclipse.paho.client.mqttv3.persist.MemoryPersistence
import com.google.gson.JsonObject
import com.google.gson.JsonParser

import org.apache.spark.storage.StorageLevel
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.streaming.mqtt._
import org.apache.spark.SparkConf

import java.util.Map.Entry
import com.google.gson.JsonElement
import java.util.Set

import scala.collection.mutable.Map
import scala.collection.mutable.ArrayBuffer

import org.apache.spark.sql._

import com.datastax.spark.connector._

import breeze.linalg._

import com.datastax.spark.connector._
import com.datastax.spark.connector.cql._

import org.apache.spark.SparkContext
import scala.math._

In [ ]:
class PrototypeNode(size:Int) extends Serializable {
         
         
  var  _nodeid : Int = -1
    
  var _position : Array[Double] = new Array[Double](size)

  var  _distance : Double = 0.0

    
  def nodeid_= (value:Int):Unit = _nodeid = value  
  def nodeid = _nodeid


  def distance_= (value:Double):Unit = _distance = value  
  def distance = _distance  


  def position_= (value:Array[Double]):Unit = _position = value  
  def position = _position



  override def toString: String =
    {
        
        var contents = "\n\t\t\t<PrototypeNode>"+_nodeid.toString+"<\\PrototypeNode>\n\t\t\t<Position>"+_position.toArray.mkString(",")+"<\\Position>\n\t\t\t<Distance>"+_distance.toString+"<\\Distance>\n"
        
       
        
        contents
    }
    
  
}

class Space() extends Serializable {

    var _nodes = ArrayBuffer[PrototypeNode]()
    def nodes_= (value:PrototypeNode):Unit = _nodes += value  
    def nodes = _nodes
    
    override def toString: String =
    {
        
        var contents = "<Map>"
        
        _nodes.foreach{node=>
        
         contents = contents+node.toString()
        
        }
        
        var closingtags = "<\\Map>"
        var value = contents+closingtags
        
        value
    }
    
}

var map = new Space()
var distances = ArrayBuffer[Double]()
var average_distance = 0.0

@transient val ssc = new StreamingContext(sc, Seconds(1))
ssc.checkpoint(".")

@transient val lines = MQTTUtils.createStream(ssc,
"ssl://hbp995.messaging.internetofthings.ibmcloud.com:8883", // Watson IoT Platform URL
                "iot-2/type/+/id/+/evt/+/fmt/+",                                // MQTT topic to receive the events
                "a:BLANK:random",                                          // Unique ID of the application
                "a-BLANK",                                                    // API-Key
                "BLANK  

@transient  val deviceMappedLines = lines.map(x => ((x.split(" ", 2)(0)).split("/")(4), x.split(" ", 2)(1)))
deviceMappedLines.print()

val batch = deviceMappedLines.map(x => {
    var readings = ArrayBuffer[Double]()
    var clean = x._2.replace("\\", "")
    clean = clean.substring(1,clean.length-1)
	val payload = new JsonParser().parse(clean).getAsJsonObject()
    val deviceObject = payload.get("d").getAsJsonObject()
	val setObj = deviceObject.entrySet()
    val itr = setObj.iterator()
    while(itr.hasNext()) {
		val entry = itr.next();
        var value = entry.getValue().getAsString()
        if(value contains "Null")
        {
          value = "0"
        }
        readings += BigDecimal(entry.getValue().getAsDouble).setScale(2, BigDecimal.RoundingMode.HALF_UP).toDouble


    }
    
	readings
})

var iteration = 1
batch.foreachRDD(rdd => rdd.collect().foreach{reading =>
      var training_phase_1 = 50
      var training_phase_2 = 50
      var currentinput = new PrototypeNode(9)
      currentinput.position = reading.toArray
      var k = 2
      if(iteration <= training_phase_1)
      {
        map.nodes = currentinput
      }
      else if (iteration <= training_phase_2)
      {
         var neighborhood = sc.parallelize(map.nodes.toSeq)
         val calculatednodes = neighborhood.map(node =>{
            
                     var nodeposition = new DenseVector(node.position)
                     var inputposition = new DenseVector(currentinput.position)
                     val squaredDistance = sum((inputposition - nodeposition):^ 2.0)  
                     currentinput.distance =  BigDecimal(squaredDistance).setScale(2, BigDecimal.RoundingMode.HALF_UP).toDouble 

             currentinput
         })
        
        val sorted_calculatedneurons = calculatednodes.takeOrdered(k)(Ordering.by(node  => node.distance)) 

          
        sorted_calculatedneurons.foreach{node=>
            
            distances += node.distance
        }
      }
      else if (iteration == training_phase_2+1)
      {
        var neighborhood = sc.parallelize(distances.toSeq)
        average_distance = neighborhood.reduce((a, b) => a + b) / distances.size
      }
      if(iteration > training_phase_2+1)
      {
         var neighborhood = sc.parallelize(map.nodes.toSeq)
         val calculatednodes = neighborhood.map(node =>{
            
                     var nodeposition = new DenseVector(node.position)
                     var inputposition = new DenseVector(currentinput.position)
                     val squaredDistance = sum((inputposition - nodeposition):^ 2.0)  
                     currentinput.distance =  BigDecimal(squaredDistance).setScale(2, BigDecimal.RoundingMode.HALF_UP).toDouble 

             currentinput
         })

          val sorted_calculatedneurons = calculatednodes.takeOrdered(1)(Ordering.by(node  => node.distance))  
          if(sorted_calculatedneurons(0).distance > average_distance)
          {
              println("Outlier Detected:")
              print("Current Sensor Reading: ")
              println(currentinput.position.toString)
              println("Nearest Neighbor: ")
              println(sorted_calculatedneurons(0).position.toString)
              println("Distance between sensor reading and nearest neighbor")
              println(sorted_calculatedneurons(0).distance)
          }
      }

})


ssc.start()
ssc.awaitTermination()
println("Finished")

-------------------------------------------
Time: 1493886884000 ms
-------------------------------------------

-------------------------------------------
Time: 1493886885000 ms
-------------------------------------------

-------------------------------------------
Time: 1493886886000 ms
-------------------------------------------

-------------------------------------------
Time: 1493886887000 ms
-------------------------------------------
(8649865433,"{\"d\" : {\"temp\" : \"11.92\" , \"cond\" : \"51.53\" , \"ph\" : \"6.87\" , \"depth\" : \"1.29\" , \"tur\" : \"7.38\" , \"dosat\" : \"7.38\" , \"domgl\" : \"10.74\" , \"power\" : \"11.19\" , \"latitude\" : \"33.46606\" , \"longitude\" : \"-81.93111\" , \"meantempm\" : \"9.0\" , \"precipm\" : \"0.0\" } }")

-------------------------------------------
Time: 1493886888000 ms
-------------------------------------------
(8649865433,"{\"d\" : {\"temp\" : \"11.89\" , \"cond\" : \"51.59\" , \"ph\" : \"6.86\" , \"depth\" : \"1.29\" , \"tur\" :